Install neccessary packages 

In [2]:
from urllib import request as rq
import pandas as pd

Download the Dublin Bus data from https://data.gov.ie/organization/dublin-city-council?tags=Transport+and+Infrastructure

In [3]:
url = "https://opendata.dublincity.ie/TrafficOpenData/sir010113-310113.zip"
busFile = rq.urlretrieve(url, './Zips/Bus/DublinBusdata.zip' )

In [38]:
import zipfile as zip

with zip.ZipFile(busFile[0],  mode='r') as arc: 
    arc.extractall('./Data/Bus/Gz/')

Use a generator to load teh data into pandas data frame https://pandasninja.com/2019/04/how-to-read-lots-of-csv-files-easily-into-pandas/#:~:text=How%20to%20read%20lots%20of%20csv%20files%20easily,need%20...%204%204.%20Leverage%20regular%20expressions%20


In [3]:
import os 
files = os.listdir('./Data/Bus/Gz/')
DBfiles = [f for f in files if f.endswith('.gz')]
def load_Files(files):
    for f in files:
        yield pd.read_csv('./Data/Bus/Gz/' + f, compression='gzip')

df = pd.concat(load_Files(DBfiles))

Download the weather station details https://cli.fusio.net/cli/climate_data/webdata/StationDetails.csv

{'.', '0', '1', '2', '3', 'c', 'g', 'i', 'r', 's', 'v', 'z'}

In [5]:
url = 'https://cli.fusio.net/cli/climate_data/webdata/StationDetails.csv'
weatherStationsfile = rq.urlretrieve(url, './Data/Weather/StationDetails.csv')

Load the weather statio details for Dublin

In [14]:
weatherStations = pd.read_csv(weatherStationsfile[0])
pd.head(weatherStations)

AttributeError: module 'pandas' has no attribute 'head'